In [29]:
"""
COMS W4705 - Natural Language Processing
Homework 2 - Parsing with Context Free Grammars 
Yassine Benajiba
"""

from cmath import e
import sys
from collections import defaultdict
from math import fsum

class Pcfg(object): 
    """
    Represent a probabilistic context free grammar. 
    """

    def __init__(self, grammar_file): 
        self.rhs_to_rules = defaultdict(list)
        self.lhs_to_rules = defaultdict(list)
        self.startsymbol = None 
        self.read_rules(grammar_file)      
 
    def read_rules(self,grammar_file):
        
        for line in grammar_file: 
            line = line.strip()
            if line and not line.startswith("#"):
                if "->" in line: 
                    rule = self.parse_rule(line.strip())
                    lhs, rhs, prob = rule
                    self.rhs_to_rules[rhs].append(rule)
                    self.lhs_to_rules[lhs].append(rule)
                else: 
                    startsymbol, prob = line.rsplit(";")
                    self.startsymbol = startsymbol.strip()
                    
     
    def parse_rule(self,rule_s):
        lhs, other = rule_s.split("->")
        lhs = lhs.strip()
        rhs_s, prob_s = other.rsplit(";",1) 
        prob = float(prob_s)
        rhs = tuple(rhs_s.strip().split())
        return (lhs, rhs, prob)

    def verify_grammar(self):
        """
        Return True if the grammar is a valid PCFG in CNF.
        Otherwise return False. 
        """

        """
        Rule constraints:
        - all values in lhs should add up to 1.
        - all nonterminal symbols are upper-case: 
            1. if two values on rhs, then should be no nonterminal with lower-case
            2. if one value on rhs, than this must be terminal in lower-case

        """
        # TODO, Part 1
        for lhs, rhs_probs in self.lhs_to_rules.items():
            probs = []
            for rhs_prob in rhs_probs:
                rhs = rhs_prob[1]
                probs.append(rhs_prob[2])
                if len(rhs) == 1:
                    if not rhs[0].islower():
                        print('rhs with 1 value {} not in lower case for lhs {}'.format(rhs[0], rhs_prob[0]))
                        return False
                elif len(rhs) == 2:
                    if not rhs[0].isupper() or not rhs[1].isupper():
                        print('rhs with 2 values {} {} not in lower case for lhs {}'.format(rhs[0], rhs[1], rhs_prob[0]))
                        return False
                else:
                    return False
            sum_prob = fsum(rhs_prob[2] for rhs_prob in rhs_probs)
            if (1 - sum_prob) > 0.000001:
                print('sum probabiliy of lhs {} is not 1, but {}'.format(lhs, sum_prob))
                return False
        return True


if __name__ == "__main__":
    with open('atis3.pcfg','r') as grammar_file:
        grammar = Pcfg(grammar_file)
        print(grammar.startsymbol)
        print(grammar.lhs_to_rules[grammar.startsymbol])
        if grammar.verify_grammar():
            print('Grammar is a valid PCFG in CNF.')
        else:
            print('Error. Grammar is not valid PCFG in CNF.')
            exit(0)
        


TOP
[('TOP', ('ADJP', 'PUN'), 0.00579150579151), ('TOP', ('FRAG', 'PUN'), 0.0559845559846), ('TOP', ('INTJ', 'PUN'), 0.00579150579151), ('TOP', ('NP', 'PUN'), 0.23166023166), ('TOP', ('PP', 'PUN'), 0.00965250965251), ('TOP', ('S', 'PUN'), 0.0965250965251), ('TOP', ('SBARQ', 'PUN'), 0.22972972973), ('TOP', ('SQ', 'PUN'), 0.0501930501931), ('TOP', ('VP', 'PUN'), 0.243243243243), ('TOP', ('WHNP', 'PUN'), 0.0714285714286)]
rhs with 1 value 0 not in lower case for lhs WHNP
Error. Grammar is not valid PCFG in CNF.


### Part 1 -  reading the grammer and getting started

In [2]:
with open('atis3.pcfg','r') as grammar_file: grammar = Pcfg(grammar_file)

In [4]:
grammar.startsymbol

'TOP'

In [1]:
"""
COMS W4705 - Natural Language Processing
Homework 2 - Parsing with Context Free Grammars 
Yassine Benajiba
"""
import math
import sys
from collections import defaultdict
import itertools
from grammar import Pcfg

### Use the following two functions to check the format of your data structures in part 3 ###
def check_table_format(table):
    """
    Return true if the backpointer table object is formatted correctly.
    Otherwise return False and print an error.  
    """
    if not isinstance(table, dict): 
        sys.stderr.write("Backpointer table is not a dict.\n")
        return False
    for split in table: 
        if not isinstance(split, tuple) and len(split) ==2 and \
          isinstance(split[0], int)  and isinstance(split[1], int):
            sys.stderr.write("Keys of the backpointer table must be tuples (i,j) representing spans.\n")
            return False
        if not isinstance(table[split], dict):
            sys.stderr.write("Value of backpointer table (for each span) is not a dict.\n")
            return False
        for nt in table[split]:
            if not isinstance(nt, str): 
                sys.stderr.write("Keys of the inner dictionary (for each span) must be strings representing nonterminals.\n")
                return False
            bps = table[split][nt]
            if isinstance(bps, str): # Leaf nodes may be strings
                continue 
            if not isinstance(bps, tuple):
                sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Incorrect type: {}\n".format(bps))
                return False
            if len(bps) != 2:
                sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Found more than two backpointers: {}\n".format(bps))
                return False
            for bp in bps: 
                if not isinstance(bp, tuple) or len(bp)!=3:
                    sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Backpointer has length != 3.\n".format(bp))
                    return False
                if not (isinstance(bp[0], str) and isinstance(bp[1], int) and isinstance(bp[2], int)):
                    print(bp)
                    sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Backpointer has incorrect type.\n".format(bp))
                    return False
    return True

def check_probs_format(table):
    """
    Return true if the probability table object is formatted correctly.
    Otherwise return False and print an error.  
    """
    if not isinstance(table, dict): 
        sys.stderr.write("Probability table is not a dict.\n")
        return False
    for split in table: 
        if not isinstance(split, tuple) and len(split) ==2 and isinstance(split[0], int) and isinstance(split[1], int):
            sys.stderr.write("Keys of the probability must be tuples (i,j) representing spans.\n")
            return False
        if not isinstance(table[split], dict):
            sys.stderr.write("Value of probability table (for each span) is not a dict.\n")
            return False
        for nt in table[split]:
            if not isinstance(nt, str): 
                sys.stderr.write("Keys of the inner dictionary (for each span) must be strings representing nonterminals.\n")
                return False
            prob = table[split][nt]
            if not isinstance(prob, float):
                sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a float.{}\n".format(prob))
                return False
            if prob > 0:
                sys.stderr.write("Log probability may not be > 0.  {}\n".format(prob))
                return False
    return True



class CkyParser(object):
    """
    A CKY parser.
    """

    def __init__(self, grammar): 
        """
        Initialize a new parser instance from a grammar. 
        """
        self.grammar = grammar

    def is_in_language(self,tokens):
        """
        Membership checking. Parse the input tokens and return True if 
        the sentence is in the language described by the grammar. Otherwise
        return False
        """
        # TODO, part 2
        return False 
       
    def parse_with_backpointers(self, tokens):
        """
        Parse the input tokens and return a parse table and a probability table.
        """
        # TODO, part 3
        table= None
        probs = None
        return table, probs


def get_tree(chart, i,j,nt): 
    """
    Return the parse-tree rooted in non-terminal nt and covering span i,j.
    """
    # TODO: Part 4
    return None 
 
       
if __name__ == "__main__":
    
    with open('atis3.pcfg','r') as grammar_file: 
        grammar = Pcfg(grammar_file) 
        parser = CkyParser(grammar)
        toks =['flights', 'from','miami', 'to', 'cleveland','.'] 
        #print(parser.is_in_language(toks))
        #table,probs = parser.parse_with_backpointers(toks)
        #assert check_table_format(chart)
        #assert check_probs_format(probs)
 

In [29]:
grammar.lhs_to_rules['PP']


[('PP', ('ABOUT', 'NP'), 0.00133511348465),
 ('PP', ('ADVP', 'PPBAR'), 0.00133511348465),
 ('PP', ('AFTER', 'NP'), 0.0253671562083),
 ('PP', ('AROUND', 'NP'), 0.00667556742323),
 ('PP', ('AS', 'ADJP'), 0.00133511348465),
 ('PP', ('AT', 'NP'), 0.0106809078772),
 ('PP', ('AT', 'PPBAR'), 0.00133511348465),
 ('PP', ('BEFORE', 'NP'), 0.0146862483311),
 ('PP', ('BETWEEN', 'NP'), 0.0160213618158),
 ('PP', ('BY', 'NP'), 0.00267022696929),
 ('PP', ('DURING', 'NP'), 0.00133511348465),
 ('PP', ('FOR', 'NP'), 0.00934579439252),
 ('PP', ('FROM', 'NP'), 0.332443257677),
 ('PP', ('IN', 'NP'), 0.0587449933244),
 ('PP', ('INTO', 'NP'), 0.00133511348465),
 ('PP', ('NP', 'PPBAR'), 0.00133511348465),
 ('PP', ('OF', 'NP'), 0.0520694259012),
 ('PP', ('ON', 'NP'), 0.110814419226),
 ('PP', ('PP', 'PP'), 0.00133511348465),
 ('PP', ('PP', 'PPBAR'), 0.00133511348465),
 ('PP', ('THAN', 'NP'), 0.00267022696929),
 ('PP', ('THROUGH', 'NP'), 0.00133511348465),
 ('PP', ('TO', 'ADVP'), 0.00133511348465),
 ('PP', ('TO',

In [17]:
grammar.lhs_to_rules['PP']

[('PP', ('ABOUT', 'NP'), 0.00133511348465),
 ('PP', ('ADVP', 'PPBAR'), 0.00133511348465),
 ('PP', ('AFTER', 'NP'), 0.0253671562083),
 ('PP', ('AROUND', 'NP'), 0.00667556742323),
 ('PP', ('AS', 'ADJP'), 0.00133511348465),
 ('PP', ('AT', 'NP'), 0.0106809078772),
 ('PP', ('AT', 'PPBAR'), 0.00133511348465),
 ('PP', ('BEFORE', 'NP'), 0.0146862483311),
 ('PP', ('BETWEEN', 'NP'), 0.0160213618158),
 ('PP', ('BY', 'NP'), 0.00267022696929),
 ('PP', ('DURING', 'NP'), 0.00133511348465),
 ('PP', ('FOR', 'NP'), 0.00934579439252),
 ('PP', ('FROM', 'NP'), 0.332443257677),
 ('PP', ('IN', 'NP'), 0.0587449933244),
 ('PP', ('INTO', 'NP'), 0.00133511348465),
 ('PP', ('NP', 'PPBAR'), 0.00133511348465),
 ('PP', ('OF', 'NP'), 0.0520694259012),
 ('PP', ('ON', 'NP'), 0.110814419226),
 ('PP', ('PP', 'PP'), 0.00133511348465),
 ('PP', ('PP', 'PPBAR'), 0.00133511348465),
 ('PP', ('THAN', 'NP'), 0.00267022696929),
 ('PP', ('THROUGH', 'NP'), 0.00133511348465),
 ('PP', ('TO', 'ADVP'), 0.00133511348465),
 ('PP', ('TO',

In [30]:
grammar.rhs_to_rules[('NP','VP')]

[('NP', ('NP', 'VP'), 0.00602409638554),
 ('S', ('NP', 'VP'), 0.694915254237),
 ('SBAR', ('NP', 'VP'), 0.166666666667),
 ('SQBAR', ('NP', 'VP'), 0.289156626506)]

In [24]:
x = "as"

In [10]:
import grammar
grammar = grammar.Pcfg(open('atis3.pcfg','r'))

In [38]:
list(grammar.rhs_to_rules)

[('ASEARLY', 'PP'),
 ('INTERESTED', 'PP'),
 ('LESS', 'PP'),
 ('ADVP', 'PP'),
 ('BACK', 'PP'),
 ('NP', 'APART'),
 ('ADVP', 'NP'),
 ('NP', 'FRAGBAR'),
 ('NP', 'NP'),
 ('NP', 'PP'),
 ('PP', 'FRAGBAR'),
 ('PP', 'PP'),
 ('PP', 'VP'),
 ('WHNP', 'FRAGBAR'),
 ('X', 'NP'),
 ('X', 'PP'),
 ('PP', 'ADVP'),
 ('PP', 'NP'),
 ('A', 'FLIGHT'),
 ('A', 'NPBAR'),
 ('A', 'RESERVATION'),
 ('A', 'SNACK'),
 ('A', 'STOP'),
 ('AFTERNOON', 'FLIGHTS'),
 ('ALASKA', "'S"),
 ('ALL', 'DETAILS'),
 ('ALL', 'FARES'),
 ('ALL', 'FLIGHTS'),
 ('ALL', 'NPBAR'),
 ('AMERICAN', "'S"),
 ('AMERICAN', 'AIRLINES'),
 ('AMERICAN', 'NPBAR'),
 ('ANY', 'FLIGHTS'),
 ('ANY', 'NPBAR'),
 ('ANY', 'STOPOVERS'),
 ('BUSINESS', 'NPBAR'),
 ('CHEAPEST', 'AIRFARE'),
 ('CHEAPEST', 'FARE'),
 ('CHEAPEST', 'FLIGHT'),
 ('CHEAPEST', 'NPBAR'),
 ('CLEVELAND', 'OHIO'),
 ('COACH', 'CLASS'),
 ('COACH', 'NPBAR'),
 ('D', 'C'),
 ('DAILY', 'FLIGHTS'),
 ('DELTA', 'NPBAR'),
 ('DINNER', 'FLIGHTS'),
 ('DIRECT', 'FLIGHT'),
 ('DIRECT', 'FLIGHTS'),
 ('DULLES', 'INTERNAT

In [39]:
grammar.rhs_to_rules[('ASEARLY', 'PP')][0]
# list(grammar.lhs_to_rules['PP'])

('ADJP', ('ASEARLY', 'PP'), 0.0588235294118)

In [40]:
grammar.rhs_to_rules

defaultdict(list,
            {('ASEARLY', 'PP'): [('ADJP', ('ASEARLY', 'PP'), 0.0588235294118)],
             ('INTERESTED',
              'PP'): [('ADJP', ('INTERESTED', 'PP'), 0.0588235294118)],
             ('LESS', 'PP'): [('ADJP', ('LESS', 'PP'), 0.117647058824)],
             ('ADVP', 'PP'): [('ADVP', ('ADVP', 'PP'), 0.0384615384615)],
             ('BACK', 'PP'): [('ADVP', ('BACK', 'PP'), 0.0384615384615)],
             ('NP', 'APART'): [('ADVP', ('NP', 'APART'), 0.153846153846)],
             ('ADVP', 'NP'): [('FRAG', ('ADVP', 'NP'), 0.0344827586207),
              ('NPBAR', ('ADVP', 'NP'), 0.00428265524625),
              ('VPBAR', ('ADVP', 'NP'), 0.00602409638554)],
             ('NP', 'FRAGBAR'): [('FRAG', ('NP', 'FRAGBAR'), 0.206896551724),
              ('FRAGBAR', ('NP', 'FRAGBAR'), 0.153846153846)],
             ('NP', 'NP'): [('FRAG', ('NP', 'NP'), 0.103448275862),
              ('FRAGBAR', ('NP', 'NP'), 0.0769230769231),
              ('NP', ('NP', 'NP'), 0.0102409638

In [41]:
toks =['flights', 'from', 'miami', 'to', 'cleveland', '.']

In [46]:
rules = list(grammar.rhs_to_rules)
n = len(toks)
table = [[[] for i in range(n+1)] for j in range(n+1)]

In [66]:
for i in range(n):
            for rule in rules:
                if rule == (toks[i],):
                    table[i][i+1] = set()
                    for x in grammar.rhs_to_rules[rule]:
                        table[i][i+1].add(x[0])

In [78]:
grammar.rhs_to_rules[('ASEARLY', 'PP')]

[('ADJP', ('ASEARLY', 'PP'), 0.0588235294118)]

In [55]:
for length in range(2, n+1): 
            for i in range(n-length+1): 
                j = i+length 
                for k in range (i+1, j): 
                    if not table[i][j]:
                        table[i][j] = set()
                    for rule in rules:
                        if len(rule) == 2:
                            for symbol1 in table[i][k]:
                                for symbol2 in table[k][j]:
                                    if symbol1 == rule[0] and symbol2 == rule[1]:
                                        for x in grammar.rhs_to_rules[rule]:
                                            table[i][j].add(x[0])

In [53]:
table

[[[], {'FLIGHTS', 'NP'}, [], [], [], [], []],
 [[], [], {'FROM', 'PP'}, [], [], [], []],
 [[], [], [], {'NP'}, [], [], []],
 [[], [], [], [], {'TO', 'X'}, [], []],
 [[], [], [], [], [], {'CLEVELAND', 'NP'}, []],
 [[], [], [], [], [], [], {'PUN'}],
 [[], [], [], [], [], [], []]]

In [92]:
grammar.rhs_to_rules[('NP', 'NP')][0]

('FRAG', ('NP', 'NP'), 0.103448275862)

In [58]:
for i in range(n+1):
            bleh = list()
            for j in range(n+1):
                bleh.append(table[i][j])
            print("{}, {}".format(i, bleh))

0, [[], {'FLIGHTS', 'NP'}, {'VPBAR', 'FRAG', 'SQBAR', 'NP'}, {'VPBAR', 'FRAGBAR', 'SQBAR', 'NPBAR', 'FRAG', 'NP'}, set(), {'VPBAR', 'FRAGBAR', 'SQBAR', 'NPBAR', 'FRAG', 'NP'}, {'TOP'}]
1, [[], [], {'FROM', 'PP'}, {'VPBAR', 'FRAGBAR', 'PP', 'NPBAR'}, set(), {'VPBAR', 'FRAGBAR', 'SQBAR', 'PP', 'NPBAR', 'FRAG', 'WHNPBAR'}, {'TOP'}]
2, [[], [], [], {'NP'}, set(), {'VPBAR', 'FRAG', 'SQBAR', 'NP'}, {'TOP'}]
3, [[], [], [], [], {'TO', 'X'}, {'FRAG', 'PP'}, {'TOP'}]
4, [[], [], [], [], [], {'NP', 'CLEVELAND'}, {'TOP'}]
5, [[], [], [], [], [], [], {'PUN'}]
6, [[], [], [], [], [], [], []]


In [93]:
table[0][n-1]

{'FRAG', 'FRAGBAR', 'NP', 'NPBAR', 'SQBAR', 'VPBAR'}

In [57]:
print(table)

[[[], {'FLIGHTS', 'NP'}, {'VPBAR', 'FRAG', 'SQBAR', 'NP'}, {'VPBAR', 'FRAGBAR', 'SQBAR', 'NPBAR', 'FRAG', 'NP'}, set(), {'VPBAR', 'FRAGBAR', 'SQBAR', 'NPBAR', 'FRAG', 'NP'}, {'TOP'}], [[], [], {'FROM', 'PP'}, {'VPBAR', 'FRAGBAR', 'PP', 'NPBAR'}, set(), {'VPBAR', 'FRAGBAR', 'SQBAR', 'PP', 'NPBAR', 'FRAG', 'WHNPBAR'}, {'TOP'}], [[], [], [], {'NP'}, set(), {'VPBAR', 'FRAG', 'SQBAR', 'NP'}, {'TOP'}], [[], [], [], [], {'TO', 'X'}, {'FRAG', 'PP'}, {'TOP'}], [[], [], [], [], [], {'NP', 'CLEVELAND'}, {'TOP'}], [[], [], [], [], [], [], {'PUN'}], [[], [], [], [], [], [], []]]


In [171]:

class CkyParser(object):
    """
    A CKY parser.
    """

    def __init__(self, grammar): 
        """
        Initialize a new parser instance from a grammar. 
        """
        self.grammar = grammar

    def is_in_language(self,tokens):
        """
        Membership checking. Parse the input tokens and return True if 
        the sentence is in the language described by the grammar. Otherwise
        return False
        """
        # TODO, part 2
        rhs_rules = list(self.grammar.rhs_to_rules)
        n = len(tokens)
        table = [[[] for i in range(n+1)] for j in range(n+1)] 

        for i in range(n):
            for rhs in rhs_rules:
                if rhs == (tokens[i],):
                    table[i][i+1] = set()
                    for rhs_rule in self.grammar.rhs_to_rules[rhs]:
                        lhs0 = rhs_rule[0]
                        table[i][i+1].add(lhs0)

        for l in range(2, n+1):
            for i in range(n-l+1):
                j = i + l
                for k in range(i+1, j):
                    if not table[i][j]:
                        table[i][j] = set()
                        for rhs in rhs_rules:
                            if len(rhs) == 2:
                                for hpoint in table[i][k]:
                                    for vpoint in table[k][j]:
                                        if hpoint == rhs[0] and vpoint == rhs[1]:
                                            for lhs_rhs_prob in self.grammar.rhs_to_rules[rhs]:
                                                table[i][j].add(lhs_rhs_prob[0])

        # check id S is in (1,n)
        if table[0][n-1]:
            return True

        return False 
    

    def parse_with_backpointers(self, tokens):
        """
        Parse the input tokens and return a parse table and a probability table.
        """
        # TODO, part 3
        rhs_rules = list(self.grammar.rhs_to_rules)
        n = len(tokens)
        table = dict()
        probs = dict()
        for i in range(n+1):
            for j in range(n+1):
                table[(i,j)] = dict()
        
        for i in range(n+1):
            for j in range(n+1):
                probs[(i,j)] = dict()
        
        for i in range(n):
            for rhs in rhs_rules:
                if rhs == (tokens[i],):
                    table[(i,i+1)] = dict()
                    probs[(i,i+1)] = dict()
                    for lhs_rhs_prob in self.grammar.rhs_to_rules[rhs]:
                        table[(i,i+1)][lhs_rhs_prob[0]] = lhs_rhs_prob[1][0]
                        probs[(i,i+1)][lhs_rhs_prob[0]] = math.log(lhs_rhs_prob[2])
        
        for l in range(2, n+1):
            for i in range(n-l+1):
                j = i + l
                for k in range(i+1, j):
                    if (i,j) not in table.keys():
                        table[(i,j)] = dict()
                        probs[(i,j)] = dict()
                    for rhs in rhs_rules:
                        if len(rhs) == 2:
                            for hpoint in table[(i,k)].keys():
                                for vpoint in table[(k,j)].keys(): 
                                    if hpoint == rhs[0] and vpoint == rhs[1]:
                                        for lhs_rhs_prob in self.grammar.rhs_to_rules[rhs]:
                                            if not lhs_rhs_prob[2] == 0: # probs not 0
                                                next = ((hpoint, i, k),(vpoint, j, k))
                                                prob = math.log(lhs_rhs_prob[2]) + probs[(i,k)][hpoint] + probs[(k,j)][vpoint]
                                                if lhs_rhs_prob[0] not in probs[(i,j)]:
                                                    table[(i,j)][lhs_rhs_prob[0]] = prob
                                                    probs[(i,j)][lhs_rhs_prob[0]] = next
                                                else:
                                                    if prob > probs[(i,j)][lhs_rhs_prob[0]]:
                                                        table[(i,j)][lhs_rhs_prob[0]] = next
                                                        probs[(i,j)][lhs_rhs_prob[0]] = prob
                                            else: # probs be 0
                                                if lhs_rhs_prob[0] not in table[(i,j)]:
                                                    table[(i,j)][lhs_rhs_prob[0]] = ((hpoint, i, k),(vpoint, j, k))
                                                    probs[(i,j)][lhs_rhs_prob[0]] = 0
 
        return table, probs

 

In [150]:
parser = CkyParser(grammar)
parser.is_in_language(toks)

True

In [151]:
toks =['miami', 'flights','cleveland', 'from', 'to','.']
parser.is_in_language(toks)

False

In [173]:
import math
parser = CkyParser2(grammar)
toks =['flights', 'from', 'miami', 'to', 'cleveland', '.']
table, probs = parser.parse_with_backpointers(toks)

(0, 0) {}
(0, 1) {'FLIGHTS': 'flights', 'NP': 'flights'}
(0, 2) {'FRAG': (('NP', 0, 1), ('PP', 1, 2)), 'NP': (('NP', 0, 1), ('PP', 1, 2)), 'SQBAR': (('NP', 0, 1), ('PP', 1, 2)), 'VPBAR': (('NP', 0, 1), ('PP', 1, 2))}
(0, 3) {'FRAG': (('NP', 0, 1), ('PP', 1, 3)), 'FRAGBAR': (('NP', 0, 1), ('FRAGBAR', 1, 3)), 'NP': (('NP', 0, 1), ('PP', 1, 3)), 'SQBAR': (('NP', 0, 1), ('PP', 1, 3)), 'VPBAR': (('NP', 0, 1), ('PP', 1, 3)), 'NPBAR': (('NP', 0, 1), ('NPBAR', 1, 3))}
(0, 4) {}
(0, 5) {'FRAG': (('NP', 0, 1), ('FRAGBAR', 1, 5)), 'FRAGBAR': (('NP', 0, 1), ('FRAGBAR', 1, 5)), 'NP': (('NP', 0, 1), ('NPBAR', 1, 5)), 'SQBAR': (('NP', 0, 1), ('SQBAR', 1, 5)), 'VPBAR': (('NP', 0, 1), ('VPBAR', 1, 5)), 'NPBAR': (('NP', 0, 1), ('NPBAR', 1, 5))}
(0, 6) {'TOP': (('NP', 0, 5), ('PUN', 5, 6))}
(1, 0) {}
(1, 1) {}
(1, 2) {'FROM': 'from', 'PP': 'from'}
(1, 3) {'FRAGBAR': (('PP', 1, 2), ('NP', 2, 3)), 'NPBAR': (('PP', 1, 2), ('NP', 2, 3)), 'VPBAR': (('PP', 1, 2), ('NP', 2, 3)), 'PP': (('FROM', 1, 2), ('NP', 2,

In [152]:
probs

{(0, 0): {},
 (0, 1): {'FLIGHTS': 0.0, 'NP': -3.195065176174159},
 (0, 2): {'FRAG': (('NP', 0, 1), ('PP', 2, 1)),
  'NP': (('NP', 0, 1), ('PP', 2, 1)),
  'SQBAR': (('NP', 0, 1), ('PP', 2, 1)),
  'VPBAR': (('NP', 0, 1), ('PP', 2, 1))},
 (0, 3): {},
 (0, 4): {},
 (0, 5): {},
 (0, 6): {},
 (1, 0): {},
 (1, 1): {},
 (1, 2): {'FROM': 0.0, 'PP': -6.618738983514369},
 (1, 3): {'FRAGBAR': (('PP', 1, 2), ('NP', 3, 2)),
  'NPBAR': (('PP', 1, 2), ('NP', 3, 2)),
  'VPBAR': (('PP', 1, 2), ('NP', 3, 2)),
  'PP': (('FROM', 1, 2), ('NP', 3, 2))},
 (1, 4): {},
 (1, 5): {},
 (1, 6): {},
 (2, 0): {},
 (2, 1): {},
 (2, 2): {},
 (2, 3): {'NP': -4.706522680248739},
 (2, 4): {},
 (2, 5): {},
 (2, 6): {},
 (3, 0): {},
 (3, 1): {},
 (3, 2): {},
 (3, 3): {},
 (3, 4): {'TO': 0.0, 'X': -1.2527629684963681},
 (3, 5): {'FRAG': (('X', 3, 4), ('NP', 5, 4)),
  'PP': (('TO', 3, 4), ('NP', 5, 4))},
 (3, 6): {},
 (4, 0): {},
 (4, 1): {},
 (4, 2): {},
 (4, 3): {},
 (4, 4): {},
 (4, 5): {'CLEVELAND': 0.0, 'NP': -3.98058567

In [123]:
tokens =['flights', 'from', 'miami', 'to', 'cleveland', '.']

In [125]:
rhs_rules = list(grammar.rhs_to_rules)
n = len(tokens)
table = dict()
probs = dict()
for i in range(n+1):
    for j in range(n+1):
        table[(i,j)] = dict()

for i in range(n+1):
    for j in range(n+1):
        probs[(i,j)] = dict()

for i in range(n):
    for rhs in rhs_rules:
        if rhs == (tokens[i],):
            table[(i,i+1)] = dict()
            probs[(i,i+1)] = dict()
            for lhs_rhs_prob in grammar.rhs_to_rules[rhs]:
                table[(i,i+1)][lhs_rhs_prob[0]] = lhs_rhs_prob[1][0]
                probs[(i,i+1)][lhs_rhs_prob[0]] = math.log(lhs_rhs_prob[2])



In [127]:
table;probs

{(0, 0): {},
 (0, 1): {'FLIGHTS': 0.0, 'NP': -3.195065176174159},
 (0, 2): {},
 (0, 3): {},
 (0, 4): {},
 (0, 5): {},
 (0, 6): {},
 (1, 0): {},
 (1, 1): {},
 (1, 2): {'FROM': 0.0, 'PP': -6.618738983514369},
 (1, 3): {},
 (1, 4): {},
 (1, 5): {},
 (1, 6): {},
 (2, 0): {},
 (2, 1): {},
 (2, 2): {},
 (2, 3): {'NP': -4.706522680248739},
 (2, 4): {},
 (2, 5): {},
 (2, 6): {},
 (3, 0): {},
 (3, 1): {},
 (3, 2): {},
 (3, 3): {},
 (3, 4): {'TO': 0.0, 'X': -1.2527629684963681},
 (3, 5): {},
 (3, 6): {},
 (4, 0): {},
 (4, 1): {},
 (4, 2): {},
 (4, 3): {},
 (4, 4): {},
 (4, 5): {'CLEVELAND': 0.0, 'NP': -3.9805856768644103},
 (4, 6): {},
 (5, 0): {},
 (5, 1): {},
 (5, 2): {},
 (5, 3): {},
 (5, 4): {},
 (5, 5): {},
 (5, 6): {'PUN': 0.0},
 (6, 0): {},
 (6, 1): {},
 (6, 2): {},
 (6, 3): {},
 (6, 4): {},
 (6, 5): {},
 (6, 6): {}}

In [128]:
for l in range(2, n+1):
    for i in range(n-l+1):
        j = i + l
        for k in range(i+1, j):
            if (i,j) not in table.keys():
                table[(i,j)] = dict()
                probs[(i,j)] = dict()
            for rhs in rhs_rules:
                if len(rhs) == 2:
                    for hpoint in table[(i,k)].keys():
                        for vpoint in table[(k,j)].keys(): 
                            if hpoint == rhs[0] and vpoint == rhs[1]:
                                for lhs_rhs_prob in grammar.rhs_to_rules[rhs]:
                                    if not lhs_rhs_prob[2] == 0: # probs not 0
                                        next = ((hpoint, i, k),(vpoint, j, k))
                                        prob = math.log(lhs_rhs_prob[2]) + probs[(i,k)][hpoint] + probs[(k,j)][vpoint]
                                        if lhs_rhs_prob[0] not in probs[(i,j)]:
                                            table[(i,j)][lhs_rhs_prob[0]] = prob
                                            probs[(i,j)][lhs_rhs_prob[0]] = next
                                        else:
                                            if prob > probs[(i,j)][lhs_rhs_prob[0]]:
                                                table[(i,j)][lhs_rhs_prob[0]] = next
                                                probs[(i,j)][lhs_rhs_prob[0]] = prob
                                    else: # probs be 0
                                        table[(i,j)][lhs_rhs_prob[0]] = ((hpoint, i, k),(vpoint, j, k))
                                        probs[(i,j)][lhs_rhs_prob[0]] = 0




TypeError: unsupported operand type(s) for +: 'float' and 'tuple'

In [136]:
probs[(4,5)].keys()

dict_keys(['CLEVELAND', 'NP'])

In [141]:
math.log(lhs_rhs_prob[2]) + probs[(i,k)][hpoint] + probs[(k,j)][vpoint]

TypeError: unsupported operand type(s) for +: 'float' and 'tuple'

In [148]:
probs[(k,j)]

{'FRAGBAR': (('PP', 1, 2), ('NP', 3, 2)),
 'NPBAR': (('PP', 1, 2), ('NP', 3, 2)),
 'VPBAR': (('PP', 1, 2), ('NP', 3, 2)),
 'PP': (('FROM', 1, 2), ('NP', 3, 2))}

In [101]:
grammar.rhs_to_rules

defaultdict(list,
            {('ASEARLY', 'PP'): [('ADJP', ('ASEARLY', 'PP'), 0.0588235294118)],
             ('INTERESTED',
              'PP'): [('ADJP', ('INTERESTED', 'PP'), 0.0588235294118)],
             ('LESS', 'PP'): [('ADJP', ('LESS', 'PP'), 0.117647058824)],
             ('ADVP', 'PP'): [('ADVP', ('ADVP', 'PP'), 0.0384615384615)],
             ('BACK', 'PP'): [('ADVP', ('BACK', 'PP'), 0.0384615384615)],
             ('NP', 'APART'): [('ADVP', ('NP', 'APART'), 0.153846153846)],
             ('ADVP', 'NP'): [('FRAG', ('ADVP', 'NP'), 0.0344827586207),
              ('NPBAR', ('ADVP', 'NP'), 0.00428265524625),
              ('VPBAR', ('ADVP', 'NP'), 0.00602409638554)],
             ('NP', 'FRAGBAR'): [('FRAG', ('NP', 'FRAGBAR'), 0.206896551724),
              ('FRAGBAR', ('NP', 'FRAGBAR'), 0.153846153846)],
             ('NP', 'NP'): [('FRAG', ('NP', 'NP'), 0.103448275862),
              ('FRAGBAR', ('NP', 'NP'), 0.0769230769231),
              ('NP', ('NP', 'NP'), 0.0102409638

In [159]:
class CkyParser2(object):
    """
    A CKY parser.
    """

    def __init__(self, grammar): 
        """
        Initialize a new parser instance from a grammar. 
        """
        self.grammar = grammar

    def is_in_language(self,tokens):

        rules = list(self.grammar.rhs_to_rules)
        n = len(tokens)
        table = [[[] for i in range(n+1)] for j in range(n+1)]
        
        for i in range(n):
            for rule in rules:
                if rule == (tokens[i],):
                    table[i][i+1] = set()
                    for x in self.grammar.rhs_to_rules[rule]:
                        table[i][i+1].add(x[0])
        for length in range(2, n+1): 
            for i in range(n-length+1): 
                j = i+length 
                for k in range (i+1, j): 
                    if not table[i][j]:
                        table[i][j] = set()
                    for rule in rules:
                        if len(rule) == 2:
                            for symbol1 in table[i][k]:
                                for symbol2 in table[k][j]:
                                    if symbol1 == rule[0] and symbol2 == rule[1]:
                                        for x in self.grammar.rhs_to_rules[rule]:
                                            table[i][j].add(x[0])
        for i in range(n+1):
            bleh = list()
            for j in range(n+1):
                bleh.append(table[i][j])
            print("{}, {}".format(i, bleh))
        if table[0][n-1]:
            return True     
        return False


    def parse_with_backpointers(self, tokens):

        rules = list(self.grammar.rhs_to_rules)
        n = len(tokens)
        table = dict()
        probs = dict()
        for i in range(n+1):
            for j in range(n+1): 
                table[(i,j)] = dict()
        for i in range(n+1):
            for j in range(n+1): 
                probs[(i,j)] = dict()
        
        for i in range(n):
            for rule in rules:
                if rule == (tokens[i],):
                    table[(i,i+1)] = dict()
                    probs[(i,i+1)] = dict()
                    for x in self.grammar.rhs_to_rules[rule]:
                        table[(i,i+1)][x[0]] = x[1][0]
                        probs[(i,i+1)][x[0]] = math.log(x[2])
        for length in range(2, n+1): 
            for i in range(n-length+1): 
                j = i+length 
                for k in range (i+1, j): 
                    if (i, j) not in table.keys():
                        table[(i, j)] = dict()
                        probs[(i, j)] = dict()
                    for rule in rules:
                        if len(rule) == 2:
                            #print(rule, table[(i, k)],table[(k,j)])
                            for symbol1 in table[(i, k)].keys():
                                for symbol2 in table[(k, j)].keys():
                                    if symbol1 == rule[0] and symbol2 == rule[1]:
                                        for x in self.grammar.rhs_to_rules[rule]:
                                            if not x[2] == 0:
                                                children = ((symbol1, i, k), (symbol2, k, j))
                                                probabiity = math.log(x[2]) + probs[(i, k)][symbol1] + probs[(k, j)][symbol2]
                                                if x[0] not in probs[(i, j)]:
                                                    probs[(i, j)][x[0]] = probabiity
                                                    table[(i, j)][x[0]] = children
                                                else:
                                                    current_prob = probs[(i, j)][x[0]]
                                                    if probabiity > current_prob:
                                                        probs[(i, j)][x[0]] = probabiity
                                                        table[(i, j)][x[0]] = children
                                            else:
                                                current_prob = 0
                                                if x[0] not in table[(i, j)]:
                                                    table[(i, j)][x[0]] = ((symbol1, i, k), (symbol2, k, j))
                                                    probs[(i, j)][x[0]] = current_prob
        for i in range(n+1):
            for j in range(n+1): 
                if (i,j) in table:
                    print((i,j), table[(i,j)])      

        return table, probs
    def get_tree(chart, i,j,nt): 
        """
        Return the parse-tree rooted in non-terminal nt and covering span i,j.
        """
        # TODO: Part 4
        next = chart[(i,j)][nt]

        if isinstance(next, str):
            return(nt, next)
        left = next[0]
        right = next[1]
        left_tree = get_tree(chart, left[1], left[2], left[0])
        right_tree = get_tree(chart, right[1], right[2], left[0])


        return (nt, left_tree, right_tree) 


In [156]:
import math
parser = CkyParser2(grammar)
toks =['flights', 'from', 'miami', 'to', 'cleveland', '.']
table, probs = parser.parse_with_backpointers(toks)

(0, 0) {}
(0, 1) {'FLIGHTS': 'flights', 'NP': 'flights'}
(0, 2) {'FRAG': (('NP', 0, 1), ('PP', 1, 2)), 'NP': (('NP', 0, 1), ('PP', 1, 2)), 'SQBAR': (('NP', 0, 1), ('PP', 1, 2)), 'VPBAR': (('NP', 0, 1), ('PP', 1, 2))}
(0, 3) {'FRAG': (('NP', 0, 1), ('PP', 1, 3)), 'FRAGBAR': (('NP', 0, 1), ('FRAGBAR', 1, 3)), 'NP': (('NP', 0, 1), ('PP', 1, 3)), 'SQBAR': (('NP', 0, 1), ('PP', 1, 3)), 'VPBAR': (('NP', 0, 1), ('PP', 1, 3)), 'NPBAR': (('NP', 0, 1), ('NPBAR', 1, 3))}
(0, 4) {}
(0, 5) {'FRAG': (('NP', 0, 1), ('FRAGBAR', 1, 5)), 'FRAGBAR': (('NP', 0, 1), ('FRAGBAR', 1, 5)), 'NP': (('NP', 0, 1), ('NPBAR', 1, 5)), 'SQBAR': (('NP', 0, 1), ('SQBAR', 1, 5)), 'VPBAR': (('NP', 0, 1), ('VPBAR', 1, 5)), 'NPBAR': (('NP', 0, 1), ('NPBAR', 1, 5))}
(0, 6) {'TOP': (('NP', 0, 5), ('PUN', 5, 6))}
(1, 0) {}
(1, 1) {}
(1, 2) {'FROM': 'from', 'PP': 'from'}
(1, 3) {'FRAGBAR': (('PP', 1, 2), ('NP', 2, 3)), 'NPBAR': (('PP', 1, 2), ('NP', 2, 3)), 'VPBAR': (('PP', 1, 2), ('NP', 2, 3)), 'PP': (('FROM', 1, 2), ('NP', 2,

In [169]:

### Use the following two functions to check the format of your data structures in part 3 ###
def check_table_format(table):
    """
    Return true if the backpointer table object is formatted correctly.
    Otherwise return False and print an error.  
    """
    if not isinstance(table, dict): 
        sys.stderr.write("Backpointer table is not a dict.\n")
        return False
    for split in table: 
        if not isinstance(split, tuple) and len(split) ==2 and \
          isinstance(split[0], int)  and isinstance(split[1], int):
            sys.stderr.write("Keys of the backpointer table must be tuples (i,j) representing spans.\n")
            return False
        if not isinstance(table[split], dict):
            sys.stderr.write("Value of backpointer table (for each span) is not a dict.\n")
            return False
        for nt in table[split]:
            if not isinstance(nt, str): 
                sys.stderr.write("Keys of the inner dictionary (for each span) must be strings representing nonterminals.\n")
                return False
            bps = table[split][nt]
            if isinstance(bps, str): # Leaf nodes may be strings
                continue 
            if not isinstance(bps, tuple):
                sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Incorrect type: {}\n".format(bps))
                return False
            if len(bps) != 2:
                sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Found more than two backpointers: {}\n".format(bps))
                return False
            for bp in bps: 
                if not isinstance(bp, tuple) or len(bp)!=3:
                    sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Backpointer has length != 3.\n".format(bp))
                    return False
                if not (isinstance(bp[0], str) and isinstance(bp[1], int) and isinstance(bp[2], int)):
                    print(bp)
                    sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Backpointer has incorrect type.\n".format(bp))
                    return False
    return True

def check_probs_format(table):
    """
    Return true if the probability table object is formatted correctly.
    Otherwise return False and print an error.  
    """
    if not isinstance(table, dict): 
        sys.stderr.write("Probability table is not a dict.\n")
        return False
    for split in table: 
        if not isinstance(split, tuple) and len(split) ==2 and isinstance(split[0], int) and isinstance(split[1], int):
            sys.stderr.write("Keys of the probability must be tuples (i,j) representing spans.\n")
            return False
        if not isinstance(table[split], dict):
            sys.stderr.write("Value of probability table (for each span) is not a dict.\n")
            return False
        for nt in table[split]:
            if not isinstance(nt, str): 
                sys.stderr.write("Keys of the inner dictionary (for each span) must be strings representing nonterminals.\n")
                return False
            prob = table[split][nt]
            if not isinstance(prob, float):
                sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a float.{}\n".format(prob))
                return False
            if prob > 0:
                sys.stderr.write("Log probability may not be > 0.  {}\n".format(prob))
                return False
    return True


def get_tree(chart, i,j,nt): 
    """
    Return the parse-tree rooted in non-terminal nt and covering span i,j.
    """
    # TODO: Part 4
    next = chart[(i,j)][nt]

    if isinstance(next, str):
        return(nt, next)
    left = next[0]
    right = next[1]
    left_tree = get_tree(chart, left[1], left[2], left[0])
    right_tree = get_tree(chart, right[1], right[2], left[0])

    
    return (nt, left_tree, right_tree) 
 

In [167]:
check_probs_format(probs)

True

In [170]:
get_tree(table, 0, len(toks), grammar.startsymbol)

KeyError: 'NP'